MIT License

Copyright (c) 2019 Andy Neff

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os

In [ ]:
# cut=0.3; w=0.1
cut=0.4;w=0.12
# cut=0.5;w=0.2

In [ ]:
x = [-1+cut, 1-cut, 1-cut,    -1]
y = [-1,        -1,    -1,    -1]
z = [-1,        -1, 1-cut, 1-cut]

x += [    -1,     -1,    -1]
y += [    -1,  1-cut, 1-cut]
z += [-1+cut, -1+cut,     1]

x += [    -1,  1-cut, 1-cut]
y += [-1+cut, -1+cut,     1]
z += [     1,      1,     1]

x += [-1+cut, -1+cut,      1]
y += [     1,      1,      1]
z += [     1, -1+cut, -1+cut]

x += [     1,      1,      1]
y += [     1, -1+cut, -1+cut]
z += [ 1-cut,  1-cut,     -1]

x += [    1, -1+cut, -1+cut]
y += [1-cut,  1-cut,     -1]
z += [   -1,     -1,     -1]

x = np.array(x)
y = np.array(y)
z = np.array(z)

In [ ]:
face_a = np.array([[[x[0]-w, x[1]+w, x[2]+w, x[3]-w, x[4]-w, x[4]+w, x[3]+w, x[2]-w, x[1]-w, x[0]-w, x[0]-w],
                    y[0:1] * np.ones(11) + w,
                    [z[0]-w, z[1]-w, z[2]+w, z[3]+w, z[4]-w, z[4]-w, z[3]-w, z[2]-w, z[1]+w, z[0]+w, z[0]-w]]])

face_b = face_a.copy()
face_b[0, 1, :] -= 2*w

face_a[0, 0, 0] += 2*w
face_a[0, 0, 9:] += 2*w
face_a[0, 2, 4:6] += 2*w
# face_a = np.flip(face_a, axis=2)

face_c = np.array([[x[1] * np.ones(5) + w,
                   y[1] * np.ones(5) + [-w, w, w, -w, -w],
                   [z[1]-w, z[1]-w, z[2]+w, z[2]+w, z[1]-w]]])

face_d = face_c.copy()
face_d[0,0,:] -= 2*w
face_d[0,2,:] = [z[1]+w, z[1]+w, z[2]-w, z[2]-w, z[1]+w]

face_e = np.array([[[x[2]+w, x[2]+w, x[3]-w, x[3]-w, x[2]+w],
                    y[2] * np.ones(5) + [-w, w, w, -w, -w],
                    z[2] * np.ones(5) + w]])
# face_e = np.flip(face_e, axis=2)


face_f = face_e.copy()
face_f[0,0,:] = [x[2]-w, x[2]-w, x[3]+w, x[3]+w, x[2]-w]
face_f[0,2,:] -= 2*w
# face_f = np.flip(face_f, axis=2)

In [ ]:
faces = [face_a, face_b, face_c, face_d, face_e, face_f]
faces = [face * np.ones([6,1,1]) for face in faces]
for face in faces:
    for size in range(1,6):
        face[size,0:2,:] = face[size-1,1:3,:]
        face[size,2,:] = -face[size-1,0,:]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for face in faces:
  for f in np.rollaxis(face, 0):
    ax.plot(xs=f[0,:], ys=f[1,:], zs=f[2,:])
ax.axis('equal');

In [ ]:
# This is no good due to matplotlib z order bugs
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
fig = plt.figure()
ax = Axes3D(fig)
# x = [0,1,1,0]
# y = [0,0,1,1]
# z = [0,1,0,1]
# verts = [list(zip(x,y,z))]
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)

for face in faces:
  for f in np.rollaxis(face, 0):
    poly = Poly3DCollection([f.transpose()])
    poly.set_color(np.random.random(3))
    ax.add_collection3d(poly)

In [ ]:
def writeObj(verts,faces,outFile,commentString = ''):
    file2 = open(outFile,'w')
    file2.write("#mesh written by writeObj function. \n")
    if len(commentString) >0:
        file2.write(commentString + '\n') #emsure that each newline begins with a '#'
    for v in verts:
        file2.write("v " + str(v[0]) + " " + str(v[1]) + " " + str(v[2]) + "\n"  )
    faceBool = False
    for x in range(0,len(faces)):
        f = faces[x]
        file2.write("f ")
        for vi in f:
            file2.write(str(vi+ 1) + " ")
        file2.write("\n")

In [ ]:
points = np.concatenate([np.concatenate(face, 1) for face in faces], axis=1)

In [ ]:
face_index = [list(range(x*11, (x+1)*11)) for x in range(12)]
face_index += [list(range(132+x*5, 132+(x+1)*5)) for x in range(24)]
for x in [0, 2, 4, 7, 9, 11, 13, 15, 17, 18, 20, 22, 25, 27, 29, 30, 32, 34]:
    face_index[x].reverse()
# face_index = face_index[11:31]
writeObj(zip(*points.tolist()), face_index, "eternity.obj")